In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt    #グラフ出力用module
from PIL import Image
from sklearn.model_selection import train_test_split
import glob
import os

BATCH_SIZE = 100
WEIGHT_DECAY = 0.005
LEARNING_RATE = 0.0001
EPOCH = 100
PATH = "C:\\Users\\administrator\\CenterOfTheCircle"
#TRAINPATH = "C:\\Users\\arimoto\\cnntest\\train"
#TESTPATH = "C:\\Users\\arimoto\\cnntest\\test"
#VALIDPATH = "C:\\Users\\arimoto\\cnntest\\valid"

image_size=100

imgfiles = glob.glob(PATH + "\\20200710_064807\\*.png")

datas = []
labels = []

for imgfile in imgfiles:
    # 画像読み込み
    image = Image.open(imgfile)
    # # RGB変換
    # image = image.convert('RGB')
    # リサイズ
    image = image.resize((image_size, image_size))
    # 画像から配列に変換
    data = np.asarray(image)
    file = os.path.basename(imgfile)
    file_split = [i for i in file.split('_')]
    datas.append(data)
    labels.append((int(file_split[2]), int(file_split[3])))
    
print("fin loading")
datas = np.array(datas)
labels = np.array(labels)
#print(datas)
print(labels)

fin loading
[[100 103]
 [100 109]
 [100  17]
 ...
 [ 99  78]
 [ 99  82]
 [ 99  86]]


In [2]:
# trainデータとtestデータに分割
data_train, data_test, label_train, label_test = train_test_split(
    datas,
    labels,
    random_state = 0,
    test_size = 0.2
)
print(data_train.shape, label_train.shape, data_test.shape, label_test.shape)

(7304, 100, 100) (7304, 2) (1827, 100, 100) (1827, 2)


In [3]:
# データ型の変換＆正規化
data_train = data_train.astype('float32') / 255
data_test = data_test.astype('float32') / 255

In [4]:
trans = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor()]#,torchvision.transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5))]
)

print(data_train[0])
print(label_train[0])
print(data_test[0])
print(label_test[0])

[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]
[69 43]
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]
[49 27]


In [5]:
class Mydatasets(torch.utils.data.Dataset):
    def __init__(self, datas, labels, transform = None):
        self.transform = transform
        #self.transform2 = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

        self.data = datas
        self.label = labels

        self.datanum = datas.shape[0]

    def __len__(self):
        return self.datanum

    def __getitem__(self, idx):
        i_data = self.data[idx]
        i_label = self.label[idx]
        
        #print(out_label)
        #print(type(i_label))
        out_label = np.array(i_label.astype(np.float32))
        #out_label.append(i_label.astype(np.float32))
        #print(type(out_label))

        if self.transform:
            out_data = self.transform(i_data)
            #out_label = self.transform2(out_label)

        return out_data, out_label

In [6]:
trainset = Mydatasets(datas = data_train, labels = label_train, transform = trans)
testset = Mydatasets(datas = data_test, labels = label_test, transform = trans)

In [7]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size = BATCH_SIZE,
                        shuffle = True, num_workers = 0)
testloader = torch.utils.data.DataLoader(testset, batch_size = BATCH_SIZE,
                        shuffle = False, num_workers = 0)

In [17]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2, stride=2)

        self.conv1 = nn.Conv2d(1,16,3)
        self.conv2 = nn.Conv2d(16,32,3)

        self.fc1 = nn.Linear(32 * 23 * 23, 120)
        self.fc2 = nn.Linear(120, 2)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size()[0], -1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

class MeanSquaredLoss(nn.Module):
    def __init__(self, size_average=None, reduce=None, reduction='mean'):
        #super(nn.MSELoss, self).__init__(size_average, reduce, reduction)
        self.size_average = size_average
        self.reduce = reduce
        self.reduction = reduction
    def forward(self, input, target):
        dist1 = torch.pow((input[0] - target[0]),2)
        dist2 = torch.pow((input[1] - target[1]),2)
        loss = torch.sum(dist1 + dist2)
        return loss

In [18]:
device = torch.device("cuda:0")
net = Net()
net = net.to(device)
criterion = MeanSquaredLoss()
#criterion = nn.MSELoss()
optimizer = optim.SGD(net.parameters(), lr=LEARNING_RATE, momentum=0.9, weight_decay=WEIGHT_DECAY)

train_loss_value=[]      #trainingのlossを保持するlist
train_acc_value=[]       #trainingのaccuracyを保持するlist
test_loss_value=[]       #testのlossを保持するlist
test_acc_value=[]        #testのaccuracyを保持するlist 

max_train_loss_value = 0.
max_test_loss_value = 0.

for epoch in range(EPOCH):
    print('epoch', epoch+1)    #epoch数の出力
    for (inputs, labels) in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        print("inputs:", inputs)
        outputs = net(inputs)
        print("outputs:", outputs)
        print("labels:", labels)
        
#        out_sums = []
#        lab_sums = []
#        for out in outputs:
#            out_sums.append(out[0].item()+out[1].item())
#        for lab in labels:
#            lab_sums.append(lab[0].item()+lab[1].item())
#        print(out_sums)
#        print(lab_sums)
#        crit_outputs = torch.tensor(np.array(out_sums))
#        crit_labels = torch.tensor(np.array(lab_sums))
        
        loss = criterion(outputs, labels)#crit_outputs, crit_labels)
        print("loss:", loss)
        loss.backward()
        optimizer.step()

    sum_loss = 0.0          #lossの合計
    #sum_correct = 0         #正解率の合計
    sum_total = 0           #dataの数の合計

    #train dataを使ってテストをする(パラメータ更新がないようになっている)
    for (inputs, labels) in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)

        loss = criterion(outputs, labels)
        sum_loss += loss.item()                            #lossを足していく
        #_, predicted = outputs.max(1)                      #出力の最大値の添字(予想位置)を取得
        sum_total += labels.size(0)                        #labelの数を足していくことでデータの総和を取る
        #sum_correct += (predicted == labels).sum().item()  #予想位置と実際の正解を比べ,正解している数だけ足す
    print("train mean loss={}"
            .format(sum_loss*BATCH_SIZE/len(trainloader.dataset)))  #loss出力
    train_loss_value.append(sum_loss*BATCH_SIZE/len(trainloader.dataset))  #traindataのlossをグラフ描画のためにlistに保持
    #train_acc_value.append(float(sum_correct/sum_total))   #traindataのaccuracyをグラフ描画のためにlistに保持
    if sum_loss*BATCH_SIZE/len(trainloader.dataset) > max_train_loss_value:
        max_train_loss_value = sum_loss*BATCH_SIZE/len(trainloader.dataset)

    sum_loss = 0.0
    #sum_correct = 0
    sum_total = 0

    #test dataを使ってテストをする
    for (inputs, labels) in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)

        loss = criterion(outputs, labels)
        sum_loss += loss.item()
        #_, predicted = outputs.max(1)
        sum_total += labels.size(0)
        #sum_correct += (predicted == labels).sum().item()
    print("test mean loss={}"
            .format(sum_loss*BATCH_SIZE/len(testloader.dataset)))
    test_loss_value.append(sum_loss*BATCH_SIZE/len(testloader.dataset))
    #test_acc_value.append(float(sum_correct/sum_total))
    if sum_loss*BATCH_SIZE/len(testloader.dataset) > max_test_loss_value:
        max_test_loss_value = sum_loss*BATCH_SIZE/len(testloader.dataset)

plt.figure(figsize=(6,6))      #グラフ描画用
ylim = max(max_train_loss_value, max_test_loss_value)

#以下グラフ描画
plt.plot(range(EPOCH), train_loss_value)
plt.plot(range(EPOCH), test_loss_value, c='#00ff00')
plt.xlim(0, EPOCH)
plt.ylim(0, ylim)
plt.xlabel('EPOCH')
plt.ylabel('LOSS')
plt.legend(['train loss', 'test loss'])
plt.title('loss')
plt.savefig("loss_image.png")
plt.clf()

epoch 1
inputs: tensor([[[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]],


        [[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]],


        [[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.]]],


        ...,


        [[[1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          [1., 1., 1.,  ..., 1., 1., 1.],
          ...,
          [1., 

AttributeError: 'MeanSquaredLoss' object has no attribute '_forward_pre_hooks'

In [14]:
sum_loss = 0.0
sum_total = 0
for (inputs, labels) in testloader:
    inputs, labels = inputs.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = net(inputs)
    print(outputs)
    print(labels)
    loss = criterion(outputs, labels)
    sum_loss += loss.item()
    #_, predicted = outputs.max(1)
    sum_total += labels.size(0)
        #sum_correct += (predicted == labels).sum().item()
print("test mean loss={}".format(sum_loss*BATCH_SIZE/len(testloader.dataset)))

tensor([[58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55

        [ 88.,  32.]], device='cuda:0')
tensor([[58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        

        [ 36.,  52.]], device='cuda:0')
tensor([[58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        

        [ 20.,  70.]], device='cuda:0')
tensor([[58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        

        [ 94.,  82.]], device='cuda:0')
tensor([[58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        

        [ 69.,  54.]], device='cuda:0')
tensor([[58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        

        [ 99., 101.]], device='cuda:0')
tensor([[58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        

        [ 24.,  72.]], device='cuda:0')
tensor([[58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        

        [ 96., 103.]], device='cuda:0')
tensor([[58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        

        [110.,  58.]], device='cuda:0')
tensor([[58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295],
        [58.3994, 55.3295]], device='cuda:0', grad_fn=<AddmmBackward>)
tensor([[ 17.,  78.],
        [ 51.,  88.],
        [ 43.,  63.],
        [ 25.,  98.],
        [ 85.,  90.],
        [ 19.,  60.],
        [ 75., 101.],
       